# EMT Simulation of CIGRE MV with PV Inverters and Load Step

### Run simulation

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_noLoad1_LeftFeeder_With_LoadFlow_Results/Rootnet_FULL_NE_28J17h'
filename = 'CIGRE-MV'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import dpsimpy

In [ ]:
time_step = 1e-3
final_time = 3.0

In [ ]:
sim_name = 'EMT_CIGRE_MV_withDG_withLoadStep'
sim_name_pf = sim_name + "_Powerflow"

### Powerflow for Initialization

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system_pf = reader.loadCIM(50, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
pv_active_power = 50e3 * int(int(4319.1e3 / 50e3) / 9)
pv_reactive_power = np.sqrt(np.power(pv_active_power / 1, 2) - np.power(pv_active_power, 2))

for n in range(3, 12):
    connection_node = system_pf.node('N' + str(n))
    pv = dpsimpy.sp.ph1.AvVoltageSourceInverterDQ('pv_' + connection_node.name(), 'pv_' + connection_node.name(), dpsimpy.LogLevel.debug, True)
    pv.set_parameters(sys_omega=2 * np.pi * 50, sys_volt_nom=1500, p_ref=pv_active_power, q_ref=pv_reactive_power)
    pv.set_controller_parameters(Kp_pll=0.25 / 10, Ki_pll=2 / 1000,
                                Kp_power_ctrl=0.001 / 10, Ki_power_ctrl=0.08 / 1000 ,
                                Kp_curr_ctrl=0.3 / 10, Ki_curr_ctrl=10 / 1000, omega_cutoff=2*np.pi*50)
    pv.set_filter_parameters(Lf=0.002,Cf=789.3e-6,Rf=0.1,Rc=0.1)
    pv.set_transformer_parameters(nom_voltage_end_1=20e3, nom_voltage_end_2=1500, rated_power = 5e6,
                                ratio_abs=20e3 / 1500, ratio_phase=0,resistance=0, inductance=0.928e-3)
    pv.set_initial_state_values(p_init=450000.716605, q_init=-0.577218, phi_d_init=3854.197405 * 1000, phi_q_init=-0.003737 * 1000, gamma_d_init=128.892668 * 1000, gamma_q_init=23.068682 * 1000)
    system_pf.add(pv)
    system_pf.connect_component(pv, [connection_node])


logger_pf = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger_pf.log_attribute(node.name() + '.V', 'v', node)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(1.0)
sim_pf.set_final_time(2.0)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(True)

sim_pf.add_logger(logger_pf)
sim_pf.run()


### Dynamic simulation

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
reader2 = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.info, dpsimpy.LogLevel.debug)
system_emt = reader2.loadCIM(50, files, dpsimpy.Domain.EMT, dpsimpy.PhaseType.ABC, dpsimpy.GeneratorType.NONE)

pv_active_power = 50e3 * int(int(4319.1e3 / 50e3) / 9)
pv_reactive_power = np.sqrt(np.power(pv_active_power / 1, 2) - np.power(pv_active_power, 2))

for n in range(3, 12):
    connection_node = system_emt.node('N' + str(n))
    pv = dpsimpy.emt.ph3.AvVoltageSourceInverterDQ('pv_' + connection_node.name(), 'pv_' + connection_node.name(), dpsimpy.LogLevel.debug, True)
    pv.set_parameters(sys_omega=2 * np.pi * 50, sys_volt_nom=1500, p_ref=pv_active_power, q_ref=pv_reactive_power)
    pv.set_controller_parameters(Kp_pll=0.25 / 10, Ki_pll=2 / 1000,
                                Kp_power_ctrl=0.001 / 10, Ki_power_ctrl=0.08 / 1000 ,
                                Kp_curr_ctrl=0.3 / 10, Ki_curr_ctrl=10 / 1000, omega_cutoff=2*np.pi*50)
    pv.set_filter_parameters(Lf=0.002,Cf=789.3e-6,Rf=0.1,Rc=0.1)
    pv.set_transformer_parameters(nom_voltage_end_1=20e3, nom_voltage_end_2=1500, rated_power=5e6,
                                ratio_abs=20e3 / 1500, ratio_phase=0,resistance=0, inductance=0.928e-3, omega = 2 * np.pi * 50)
    pv.set_initial_state_values(p_init=450000.716605, q_init=-0.577218, phi_d_init=3854.197405 * 1000, phi_q_init=-0.003737 * 1000, gamma_d_init=128.892668 * 1000, gamma_q_init=23.068682 * 1000)
    system_emt.add(pv)
    system_emt.connect_component(pv, [connection_node])

system_emt.init_with_powerflow(system_pf)

# log node voltages
logger_emt= dpsimpy.Logger(sim_name)
for node in system_emt.nodes:
    logger_emt.log_attribute(node.name() + '.V', 'v', node)

# log line and load currents
for comp in system_emt.components:
    if isinstance(comp, dpsimpy.emt.ph3.PiLine):
        logger_emt.log_attribute(comp.name() + '.I', 'i_intf', comp)
    if isinstance(comp, dpsimpy.emt.ph3.RXLoad):
        logger_emt.log_attribute(comp.name() + '.I', 'i_intf', comp)

# log output of PV connected at N11
pv_name = 'pv_N11'
pv = system_emt.component(pv_name)
input_names = [
    "pv_powerctrl_input_pref", "pv_powerctrl_input_qref", "pv_powerctrl_input_vcd",
    "pv_powerctrl_input_vcq", "pv_powerctrl_input_ircd", "pv_powerctrl_input_ircq"
]
logger_emt.log_attribute(input_names, 'powerctrl_inputs', pv)

state_names = [
    "pv_powerctrl_state_p", "pv_powerctrl_state_q", "pv_powerctrl_state_phid",
    "pv_powerctrl_state_phiq", "pv_powerctrl_state_gammad", "pv_powerctrl_state_gammaq"
]
logger_emt.log_attribute(state_names, 'powerctrl_states', pv)

output_names = [
    "pv_powerctrl_output_vsd", "pv_powerctrl_output_vsq"
]

logger_emt.log_attribute(output_names, 'powerctrl_outputs', pv)

logger_emt.log_attribute(pv_name + '_v_intf', 'v_intf', pv)
logger_emt.log_attribute(pv_name + '_i_intf', 'i_intf', pv)
logger_emt.log_attribute(pv_name + '_pll_output', 'pll_output', pv)
logger_emt.log_attribute(pv_name + '_vsref', 'Vsref', pv)
logger_emt.log_attribute(pv_name + '_vs', 'Vs', pv)

# load step sized in absolute terms
load_switch = dpsimpy.emt.ph3.Switch("Load_Add_Switch_N11", dpsimpy.LogLevel.debug)
connection_node = system_emt.node('N11')
resistance = np.abs(connection_node.initial_single_voltage())**2 / 1500.0e3
load_switch.set_parameters(np.identity(3) * 1e9, np.identity(3) * resistance)
load_switch.open()
system_emt.add(load_switch)
system_emt.connect_component(load_switch, [dpsimpy.emt.SimNode.gnd, system_emt.node('N11')])
logger_emt.log_attribute('switchedload_i', 'i_intf', load_switch)
load_step_event = dpsimpy.event.SwitchEvent3Ph(2  - time_step, load_switch, True)


sim_emt = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim_emt.set_system(system_emt)
sim_emt.set_time_step(time_step)
sim_emt.set_final_time(final_time)
sim_emt.set_domain(dpsimpy.Domain.EMT)
sim_emt.set_solver(dpsimpy.Solver.MNA)
sim_emt.add_event(load_step_event)
sim_emt.do_steady_state_init(False)
sim_emt.do_init_from_nodes_and_terminals(True)

sim_emt.add_logger(logger_emt)
sim_emt.run()

## Powerflow

### Read results

In [ ]:
modelName = 'EMT_CIGRE_MV_withDG_withLoadStep_Powerflow'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

## EMT

In [ ]:
modelName = 'EMT_CIGRE_MV_withDG_withLoadStep'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot results

### Voltages

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-3:] == 'V_0' and ts_name[:-2] != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.values, label=ts_name)
plt.legend()
plt.show()

### Currents

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-3:] == 'I_0':
        plt.plot(ts_obj.time, ts_obj.values, label=ts_name)
plt.legend()
plt.show()